# 2021 Western North America heat wave
#### Start date	of the event: June 25, 2021, End date of the event:	July 7, 2021

Relevant info on the web:
- [Roth et al. paper that basically did the same analysis that we want to do](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2015JF003782)
- [Link to USGS stream gauge data](https://nwis.waterdata.usgs.gov/nwis/uv?cb_00060=on&cb_00065=on&cb_63680=on&format=gif_default&site_no=12101500&period=&begin_date=2021-06-1&end_date=2021-07-10)
- [Link to USGS stream gauge info](https://waterdata.usgs.gov/monitoring-location/12092000/#parameterCode=00065&period=P365D)
- [Wikipedia page about the heat dome event](https://en.wikipedia.org/wiki/2021_Western_North_America_heat_wave)

## Step 1: Download some data using ObsPy

In [ ]:
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
from time import perf_counter
from scipy.signal import spectrogram
import numpy as np
plt.style.use('bmh')
# seaborn-colorblind
t0 = perf_counter()

client = Client("IRIS")

t1 = UTCDateTime("2021-06-15T00:00:00")
t2 = UTCDateTime("2021-07-15T00:00:00")
# t2 = UTCDateTime("2021-07-08T00:00:00")

st = client.get_waveforms("CC", "PR02", "--", "BHZ", t1, t2, attach_response=True)

pre_filt = (0.1, 0.2, 25, 49)
st.remove_response(output='VEL', pre_filt=pre_filt)
st.plot()
print(perf_counter()-t0)

In [ ]:
data = np.array([])
for tr in st:
    data=np.concatenate((data,tr.data))
data = np.array(data)

## Step 2: Make a spectrogram and plot it

In [ ]:
f, t, Sxx = spectrogram(data, fs=50.0)

In [ ]:
c=plt.pcolormesh(t/86400, f, np.log10(np.abs(Sxx)), shading='flat',vmin=-17,vmax=-13)
plt.colorbar(c)
plt.tight_layout()
plt.show()

## Step 3: Extract a particular frequency band, low-pass filter it, then make a plot
The thinking here is that this frequency band is related to water flow and/or sediment flux

In [ ]:
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d

power_timeseries = np.abs( np.sum(Sxx[60:72,:],axis=0))

dt = 60
t_deci = np.arange(dt,max(t),dt)
p_filt = gaussian_filter(power_timeseries,1000)
p_fun = interp1d(t,p_filt)
p_deci = p_fun(t_deci)

fig,ax=plt.subplots(figsize=(15,10))
plt.plot(t/86400, power_timeseries  )
plt.plot(t_deci/86400, p_deci )
# plt.ylim([0, 1e-12])
plt.title('Power in band from %f to %f Hz'%(f[60],f[72]))
plt.xlabel('Days since Jun 1, 2021 GMT')
# plt.ylim([-16,-12])
# plt.yscale('log')
plt.ylim([0, 1e-13])
plt.show()

## Step 4: Load the stream gauge data and plot it

In [ ]:
import pandas as pd
  
# read text file into pandas DataFrame
df = pd.read_csv("usgs-data.txt", sep="\t", header=28)

df["datetime"]

In [ ]:
fig,ax=plt.subplots()
plt.plot(df['Flux'].to_list())
plt.show()

## Step 5: Compare seismic and stream gauge data.
Specifically: interpolate the stream gauge data so that it is on the same time grid as the spectral energy curve, then compare the two using a cross plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load USGS data from "usgs-data.txt" file
usgs_df = pd.read_csv("usgs-data.txt", sep="\t")

# Convert the 'datetime' column to a datetime object
usgs_df['datetime'] = pd.to_datetime(usgs_df['datetime'])

# Interpolate the streamflow data
usgs_df = usgs_df.set_index('datetime').resample('15T').mean().interpolate(method='linear')

# Sample spectral energy data (replace this with your data)
spectral_energy_data = {
    'time': ['2021-02-24 15:15', '2021-02-24 15:30', '2021-02-24 15:45', '2021-02-24 16:00'],
    'energy': [0.5, 0.6, 0.7, 0.8]
}

spectral_energy_df = pd.DataFrame(spectral_energy_data)

# Merge the two DataFrames based on the time column
merged_data = pd.merge(usgs_df, spectral_energy_df, left_index=True, right_on='time', how='outer')

# Create a cross plot
plt.scatter(merged_data['Flux'], merged_data['energy'], marker='o', s=10)
plt.xlabel('Streamflow')
plt.ylabel('Spectral Energy')
plt.title('Cross Plot of Stream Gauge Data vs. Spectral Energy Curve')
plt.grid()
plt.show()

## Step 6: Find some rainfall data and compare it in a similar way.